In [34]:
import pickle
import argparse
import asyncio
import json
import os
import time
import json
import requests
from collections import defaultdict
from urllib.error import HTTPError
from urllib.parse import quote_plus
from Bio import Entrez
import numpy as np
import pandas as pd
import random
import tqdm
import openai
import os
import json
import time

from openai import OpenAI, AsyncOpenAI
Entrez.api_key = "***ENTREZ_API_KEY_REDACTED***"
api_key = "OPENAI_API_KEY_REDACTED"

json_path = "/Users/yiquntchen/Downloads/test-temp-4o-one-question-async.json"  # Replace with desired output path
with open(json_path, 'rb') as f:
    converted_dict = json.load(f)
    
openai.api_key = api_key #os.getenv("OPENAI_API_KEY")


In [71]:
# Optional: safer to store as environment variable

# --- Functions from earlier ---
def generate_negation_prompt(entry):
    prompt = f"""
You are a clinical research expert with comprehensive knowledge of systematic reviews, randomized controlled trials (RCTs), 
and observational studies. Your task is to negate specific fields in a medical evidence summary while preserving the integrity of 
the original data.

Here is the medical evidence summary you need to analyze:
{{
  "doi": "{entry['doi']}",
  "question": "{entry['question']}",
  "answer": "{entry['answer']}",
  "evidence-quality": "{entry['evidence-quality']}",
  "discrepancy": "{entry['discrepancy']}",
}}

Your objective is to negate the 'question' and 'answer' fields in this summary according to the following rules:

1. NEGATED_QUESTION:
   - If the question is in the form "if A improves B", negate it to "if A reduces B". 
   - The negtation must be *logically correct*. That is, original question and answer needs to imply that the negated question and answer are correct.
   - If the question is in another form, attempt to negate its core meaning using verb while keeping other parts of the question mostly the same.
   - If negation is not clear or possible, use "Not applicable" as the NEGATED_QUESTION.

2. NEGATED_ANSWER: 
   - If the NEGATED_QUESTION is "Not applicable", answer should be "Not applicable".
   - If the answer is "Yes", change it to "No".
   - If the answer is "No", change it to "Yes".
   - If the answer is "No Evidence", leave it unchanged.

All other fields, including the original 'question', 'discrepancy', and 'evidence-quality', must remain exactly the same.

Provide the negated summary in the following JSON format:

{{
  "doi": "[Original DOI]",
  "question": "[NEGATED_QUESTION]",
  "answer": "[NEGATED_ANSWER]",
  "original_question": [Original question],
  "original_answer": [Original answer],
  "evidence-quality": "[Original evidence-quality]",
  "discrepancy": "[Original discrepancy]",
}}

Remember to replace the placeholders in square brackets with the appropriate content from the original summary or your negated versions.
"""
    return prompt.strip()

def check_negation_validity(original_answer, negated_answer):
    if original_answer == "Yes" and negated_answer == "No":
        return True
    elif original_answer == "No" and negated_answer == "Yes":
        return True
    elif original_answer == "No Evidence" and negated_answer == "No Evidence":
        return True
    else:
        return False

def process_llm_response(original_entry, llm_response_dict):
    original_answer = original_entry["answer"]
    negated_answer = llm_response_dict["answer"]
    valid = check_negation_validity(original_answer, negated_answer)
    llm_response_dict["negation-valid"] = valid
    return llm_response_dict

# --- Call OpenAI for each prompt ---
def call_openai_with_prompt(prompt, model, client, max_retries=3):
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt},
            ],
            response_format={"type": "json_object"},
            temperature=0,
        )
            raw_content = completion.choices[0].message.content.strip()
            return json.loads(raw_content)
            # return completion.choices[0].message.content
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            time.sleep(2 ** attempt)
    raise RuntimeError("OpenAI API failed after several retries.")

# --- Final driver function ---
def generate_llm_responses_and_validate(converted_dict, model):
    llm_responses = {}
    final_dataset = []
    invalid_cases = []
    client = OpenAI(api_key=api_key)

    for key, entry in tqdm.tqdm(converted_dict.items()):
        prompt = generate_negation_prompt(entry)
        try:
            llm_response = call_openai_with_prompt(prompt, model, client)
            processed = process_llm_response(entry, llm_response)
            llm_responses[key] = processed
            final_dataset.append(processed)
            if not processed["negation-valid"]:
                invalid_cases.append(processed)
        except Exception as e:
            print(f"Failed to process key {key}: {e}")

    return final_dataset, invalid_cases, llm_responses

# sampled_data = dict(random.sample(list(converted_dict.items()), 5))
final_dataset, invalid_cases, llm_responses = generate_llm_responses_and_validate(converted_dict, 'gpt-4o-mini')

# Optional: Save results
with open("/Users/yiquntchen/Downloads/test-4o-one-negated-dataset.jsonl", "w") as f:
    for item in final_dataset:
        f.write(json.dumps(item) + "\n")

In [75]:
import json

json_path = "/Users/yiquntchen/Downloads/test-4o-one-negated-dataset.jsonl"

negated_data = []
with open(json_path, 'r') as f:  # Use 'r' instead of 'rb'
    for line in f:
        negated_data.append(json.loads(line))